In [1]:
import re 
import nltk 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer ## stemming 
from nltk.stem import WordNetLemmatizer  ## lemmatization
import joblib,os 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences 
from sklearn.preprocessing import LabelEncoder ,OneHotEncoder
from sklearn.model_selection import train_test_split

print('libraries imported 😎')

libraries imported 😎


In [2]:
train_data = open("./Data/train.txt").readlines()
val_data = open("./Data/val.txt").readlines()
test_data = open("./Data/test.txt").readlines()

In [3]:
print(len(train_data))
print(len(val_data))
print(len(test_data))

16000
2000
2000


In [4]:
complet_data = train_data + test_data + val_data 
print(len(complet_data))

20000


In [5]:
x=[]
y=[]
for temp in complet_data:
    complete = temp.split(';')
    if len(complete)==2:
        x.append(complete[0])
        y.append(complete[1].strip())

In [6]:
labels=[]
for item in y:
        if item not in labels:
            labels.append(item)
labels 

['sadness', 'anger', 'love', 'surprise', 'fear', 'joy']

In [7]:
def text_cleaning(sentences,stemming):
    cleaned_data = []
    for sentece in sentences: 
        message = sentece.lower()
        message = re.sub('[^a-z0-9 ]',"",message)
        ls_of_words = nltk.word_tokenize(message)
        ls_of_word_without_stopwords = [word for word in ls_of_words if word not in stopwords.words('english')]
        stemmed_words = [stemming.stem(word) for word in ls_of_word_without_stopwords ]
        message = " ".join(stemmed_words)
        cleaned_data.append(message)
    return cleaned_data

stem = PorterStemmer()
cleaned_data = text_cleaning(x,stemming=stem)


In [8]:
os.makedirs('models',exist_ok=True)
joblib.dump(cleaned_data,"./models/cleaned_data.lb")
print("successfully saved your cleaned data!")


successfully saved your cleaned data!


In [9]:
ls_of_stopwords = stopwords.words('english')
print(x[1])
single_message = x[1].lower()
single_message = re.sub('[^a-z0-9 ]',"",single_message)
ls_of_words = nltk.word_tokenize(single_message)  # single_message.split()
ls_of_word_without_stopwords = []
for word in ls_of_words: 
    if word not in ls_of_stopwords:
        ls_of_word_without_stopwords.append(word)
print(ls_of_word_without_stopwords)
stem = PorterStemmer()
stemmed_words = []
for word in ls_of_word_without_stopwords: 
    stemmed_words.append(stem.stem(word))
print(stemmed_words)
" ".join(stemmed_words)

i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake
['go', 'feeling', 'hopeless', 'damned', 'hopeful', 'around', 'someone', 'cares', 'awake']
['go', 'feel', 'hopeless', 'damn', 'hope', 'around', 'someon', 'care', 'awak']


'go feel hopeless damn hope around someon care awak'

In [10]:
tokenizer = Tokenizer(oov_token="<nothing>")
tokenizer.fit_on_texts(cleaned_data)
joblib.dump(tokenizer,'./models/tokenizer.lb')
print("successfully saved your tokenizer at this location : './models/tokenizer.lb'")

successfully saved your tokenizer at this location : './models/tokenizer.lb'


In [11]:
tokenizer.word_counts  


OrderedDict([('didnt', 334),
             ('feel', 21204),
             ('humili', 69),
             ('go', 1101),
             ('hopeless', 81),
             ('damn', 54),
             ('hope', 320),
             ('around', 382),
             ('someon', 335),
             ('care', 254),
             ('awak', 22),
             ('im', 3055),
             ('grab', 22),
             ('minut', 79),
             ('post', 234),
             ('greedi', 79),
             ('wrong', 152),
             ('ever', 260),
             ('nostalg', 63),
             ('fireplac', 3),
             ('know', 1192),
             ('still', 743),
             ('properti', 5),
             ('grouchi', 35),
             ('ive', 723),
             ('littl', 932),
             ('burden', 95),
             ('late', 167),
             ('wasnt', 119),
             ('sure', 310),
             ('take', 403),
             ('milligram', 1),
             ('time', 1215),
             ('recommend', 19),
             ('amoun

In [12]:
tokenized_data = tokenizer.texts_to_sequences(cleaned_data)

In [13]:

len(tokenized_data)

20000

In [14]:
cleaned_data[0]

'didnt feel humili'

In [15]:
tokenized_data[0]

[61, 2, 522]

In [16]:
len_of_message = []
for i in range(len(tokenized_data)):
    len_of_message.append(len(tokenized_data[i]))
print("Your maximum length is : ",max(len_of_message))


Your maximum length is :  35


In [17]:
print("Your maximum length is : ",max(list(map(len,tokenized_data))))

Your maximum length is :  35


In [18]:
sequences = pad_sequences(tokenized_data,maxlen=35,padding='post')

In [19]:
sequences

array([[  61,    2,  522, ...,    0,    0,    0],
       [  10,    2,  419, ...,    0,    0,    0],
       [   4, 1230,  431, ...,    0,    0,    0],
       ...,
       [   2,  194,  157, ...,    0,    0,    0],
       [ 328,    2,  175, ...,    0,    0,    0],
       [   2,    3,  916, ...,    0,    0,    0]])

In [20]:
labels=[]
for item in y:
        if item not in labels:
            labels.append(item)
label_dict = {label:i for i , label in enumerate(labels)}
label_dict

{'sadness': 0, 'anger': 1, 'love': 2, 'surprise': 3, 'fear': 4, 'joy': 5}

In [21]:
from sklearn.preprocessing import LabelEncoder ,OneHotEncoder

In [22]:
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(y)

In [23]:
Y

array([4, 4, 0, ..., 2, 2, 2], dtype=int64)

In [24]:
label_encoder.classes_

array(['anger', 'fear', 'joy', 'love', 'sadness', 'surprise'], dtype='<U8')

In [25]:
joblib.dump(label_encoder,'./models/label_encoder.lb')

['./models/label_encoder.lb']

In [26]:
sequences    # <====  X DATA  cleaned data 
Y            # <====  Y DATA label 

array([4, 4, 0, ..., 2, 2, 2], dtype=int64)

In [27]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(sequences,Y,test_size=0.15)

In [28]:
## Model training
## Model define  
# ANN , CNN , RNN  
### logistic regression ,
### DTC , RDC , 
## machine learning algorithm   | ANN   | RNN 

In [29]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import LSTM , Dense

model = Sequential([
    LSTM(units=128, input_shape=(35, 1), return_sequences=True),
    LSTM(units=64),
    Dense(units=6, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

C:\Users\tumha\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [30]:
history = model.fit(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Epoch 1/5
532/532 ━━━━━━━━━━━━━━━━━━━━ 70s 120ms/step - accuracy: 0.3227 - loss: 1.6041 - val_accuracy: 0.3417 - val_loss: 1.5562
Epoch 2/5
532/532 ━━━━━━━━━━━━━━━━━━━━ 57s 107ms/step - accuracy: 0.3335 - loss: 1.5803 - val_accuracy: 0.3393 - val_loss: 1.5578
Epoch 3/5
532/532 ━━━━━━━━━━━━━━━━━━━━ 60s 112ms/step - accuracy: 0.3456 - loss: 1.5739 - val_accuracy: 0.3413 - val_loss: 1.5581
Epoch 4/5
532/532 ━━━━━━━━━━━━━━━━━━━━ 59s 111ms/step - accuracy: 0.3380 - loss: 1.5762 - val_accuracy: 0.3433 - val_loss: 1.5573
Epoch 5/5
532/532 ━━━━━━━━━━━━━━━━━━━━ 61s 114ms/step - accuracy: 0.3447 - loss: 1.5713 - val_accuracy: 0.3433 - val_loss: 1.5614


In [31]:
prediction = model.predict(x_test)

94/94 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step


In [32]:
import numpy as np 
np.argmax(prediction,axis=0)


array([ 305,  305,  210, 2325,  305,   21], dtype=int64)

In [33]:
prediction = np.argmax(prediction,axis=1)

In [34]:
prediction

array([2, 2, 2, ..., 2, 2, 2], dtype=int64)

In [35]:
y_test

array([3, 3, 2, ..., 1, 2, 4], dtype=int64)

In [36]:
import pandas as pd 
df = pd.DataFrame({'actual':y_test,'prediction':prediction})

In [37]:
label_dictionary = {value:key for key , value in label_dict.items()}
label_dictionary

{0: 'sadness', 1: 'anger', 2: 'love', 3: 'surprise', 4: 'fear', 5: 'joy'}

In [38]:
df['actual'] = df['actual'].map(label_dictionary)
df['prediction'] = df['prediction'].map(label_dictionary)

In [39]:
df.head(40)

,actual,prediction
0,surprise,love
1,surprise,love
2,love,love
3,fear,love
4,love,love
5,love,love
6,love,love
7,fear,love
8,fear,love
9,surprise,love


In [40]:
### using RNN 
# sequential SimpleRNN model defining
from tensorflow.keras.layers import SimpleRNN
model = Sequential()
model.add(SimpleRNN(32,input_shape=(35,1),return_sequences=False))
model.add(Dense(6,activation='softmax'))



## compiling the model
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.summary()
# return_sequence = False  --> means if you don,t want to stack again this model then specify False

C:\Users\tumha\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)               │ (None, 32)                  │           1,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 6)                   │             198 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,286 (5.02 KB)

 Trainable params: 1,286 (5.02 KB)

 Non-trainable params: 0 (0.00 B)

In [41]:
## model save
import os
os.makedirs('models',exist_ok=True)  
model.save('./models/lstm_model.h5')

In [42]:
## to load the tensorflow models 
from tensorflow.keras.models import  load_model 
loaded_model = load_model('./models/lstm_model.h5')

In [43]:
loaded_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)               │ (None, 32)                  │           1,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 6)                   │             198 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,288 (5.04 KB)

 Trainable params: 1,286 (5.02 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)